# Przetwarzanie wstępne. Filtracja kontekstowa.


### Cel:
- zapoznanie z pojęciem kontekstu / filtracji kontekstowej,
- zapoznanie z pojęciem konwolucji (splotu),
- zapoznanie z wybranymi filtrami:
	- filtry liniowe dolnoprzepustowe:
		- filtr uśredniający,
		- filtr Gaussa.
	- filtry nielinowe:
		- mediana,
		- mediana dla obrazów kolorowych.
	- filtry liniowe górnoprzepustowe:
			- laplasjan,
			- operator Robersta, Prewitta, Sobela.
- zadanie domowe: adaptacyjna filtracja medianowa.

### Filtry liniowe uśredniające (dolnoprzepustowe)

Jest to podstawowa rodzina filtrów stosowana w cyfrowym przetwarzaniu obrazów. 
Wykorzystuje się je w celu "rozmazania" obrazu i tym samym redukcji szumów (zakłóceń) na obrazie.
Filtr określony jest przez dwa parametry: rozmiar maski (ang. _kernel_) oraz wartości współczynników maski.

Warto zwrócić uwagę, że omawiane w niniejszym rozdziale operacje generują nową wartość piksela na podstawie pewnego fragmentu obrazu (tj. kontekstu), a nie jak operacje punktowe tylko na podstawie jednego piksela.


1. Wczytaj obraz _plansza.png_.
W dalszej części ćwiczenia sprawdzenie działania filtracji dla innych obrazów sprowadzi się do wczytania innego pliku.

2. Podstawowa funkcja to `cv2.filter2D`  - realizacja filtracji konwolucyjnej.
   Proszę sprawdzić jej dokumentację i zwrócić uwagę na obsługę problemu brzegowego (na krawędziach istnieją piksele dla których nie da się wyznaczyć otoczenia).

  Uwaga. Problem ten można też rozwiązać z użyciem funkcji `signal.convolve2d` z biblioteki _scipy_ (`from scipy import signal`).

3. Stwórz podstawowy filtr uśredniający o rozmiarze $3 \times 3$ -- za pomocą funkcji `np.ones`. Wykonaj konwolucję na wczytanym obrazie. Na wspólnym rysunku wyświetl obraz oryginalny, po filtracji oraz moduł z różnicy.

4. Przeanalizuj otrzymane wyniki. Jakie elementy zawiera obraz "moduł z różnicy"? Co na tej podstawie można powiedzieć o filtracji dolnoprzepustowej?

In [ ]:
import matplotlib.pyplot as plt
import cv2
import os
import numpy as np
# from scipy import signal


# Obrazki
if not os.path.exists("jet.png") :
    !wget https://raw.githubusercontent.com/vision-agh/poc_sw/master/06_Context/jet.png --no-check-certificate
if not os.path.exists("kw.png") :
    !wget https://raw.githubusercontent.com/vision-agh/poc_sw/master/06_Context/kw.png --no-check-certificate
if not os.path.exists("moon.png") :
    !wget https://raw.githubusercontent.com/vision-agh/poc_sw/master/06_Context/moon.png --no-check-certificate
if not os.path.exists("lenaSzum.png") :
    !wget https://raw.githubusercontent.com/vision-agh/poc_sw/master/06_Context/lenaSzum.png --no-check-certificate
if not os.path.exists("lena.png") :
    !wget https://raw.githubusercontent.com/vision-agh/poc_sw/master/06_Context/lena.png --no-check-certificate
if not os.path.exists("plansza.png") :
    !wget https://raw.githubusercontent.com/vision-agh/poc_sw/master/06_Context/plansza.png --no-check-certificate




In [ ]:
KERNEL_DIM = 5
img = cv2.imread('plansza.png', cv2.IMREAD_GRAYSCALE)

def low_pass(img, dim):
    kernel = np.ones((dim, dim), np.float32) / (dim * dim)

    filtered = cv2.filter2D(img, - 1, kernel)

    fig, axs = plt.subplots(1, 3, figsize=(50,20))
    axs[0].imshow(img, 'gray')
    axs[0].axis('off')
    axs[0].set_title('Original', fontsize=40)

    axs[1].imshow(filtered, 'gray')
    axs[1].axis('off')
    axs[1].set_title('Filtered with KERNEL_DIM = {0}'.format(dim), fontsize=40)

    axs[2].imshow(np.abs(img - filtered), 'gray')
    axs[2].axis('off')
    axs[2].set_title('Substraction abs', fontsize=40)
  

low_pass(img, KERNEL_DIM)

5. Na wspólnym rysunku wyświetl wyniki filtracji uśredniającej z oknem o rozmiarze 3, 5, 9, 15 i 35. 
Wykorzystaj polecenie `plt.subplot`. 
Przeanalizuj wpływ rozmiaru maski na wynik. 

In [ ]:
KERNEL_DIMS = [3, 5, 9, 15, 35]
PATH = 'lena.png'

fig, axs = plt.subplots(2, len(KERNEL_DIMS) // 2 + 1, figsize=(30, 10))
img = cv2.imread(PATH, cv2.IMREAD_GRAYSCALE)
axs[0][0].imshow(img, 'gray')
axs[0][0].set_title('original',  fontsize=20)
axs[0][0].axis('off')


i, j = 1, 0
for dim in KERNEL_DIMS:
    kernel = np.ones((dim, dim), np.float32) / (dim * dim)
    filtered = cv2.filter2D(img, - 1, kernel)
    axs[j][i].imshow(filtered, 'gray')
    axs[j][i].set_title('KERNEL_DIM = {0}'.format(dim), fontsize=20)
    axs[j][i].axis('off')
    
    # indices for subploting
    i += 1
    if i == 3: 
        j += 1
        i = 0
    

6. Wczytaj obraz _lena.png_.
Zaobserwuj efekty filtracji dolnoprzepustowej dla obrazu rzeczywistego.

In [ ]:
KERNEL_DIM = 10
low_pass(cv2.imread('lena.png', cv2.IMREAD_GRAYSCALE), KERNEL_DIM)

7. Niekorzystny efekt towarzyszący wykonanym filtracjom dolnoprzepustowym to utrata ostrości. 
Częściowo można go zniwelować poprzez odpowiedni dobór maski. 
Wykorzystaj maskę:  `M = np.array([1 2 1; 2 4 2; 1 2 1])`. 
Przed obliczeniami należy jeszcze wykonać normalizację - podzielić każdy element maski przez sumę wszystkich elementów: `M = M/sum(sum(M));`.
Tak przygotowaną maskę wykorzystaj w konwolucji - wyświetl wyniki tak jak wcześniej.
Możliwe jest też wykorzystywanie innych masek - współczynniki można dopasowywać do konkretnego problemu.

In [ ]:
def low_pass_custom_mask(img, mask):
    mask = mask / sum(mask)
    filtered = cv2.filter2D(img, - 1, mask)

    fig, axs = plt.subplots(1, 3, figsize=(50,20))
    axs[0].imshow(img, 'gray')
    axs[0].axis('off')
    axs[0].set_title('Original', fontsize=40)

    axs[1].imshow(filtered, 'gray')
    axs[1].axis('off')
    axs[1].set_title('Filtered with MASK =\n {0}'.format(mask), fontsize=40)

    axs[2].imshow(np.abs(img - filtered), 'gray')
    axs[2].axis('off')
    axs[2].set_title('Substraction abs', fontsize=40)
    
    
CUSTOM_MASK = np.array([[1, 2, 1], [2, 4, 2], [1, 2, 1]])
low_pass_custom_mask(cv2.imread('lena.png', cv2.IMREAD_GRAYSCALE), CUSTOM_MASK)

8. Skuteczną i często wykorzystywaną maską jest tzw. maska Gasussa.
Jest to zbiór liczb, które aproksymują dwuwymiarowy rozkład Gaussa. 
Parametrem jest odchylenie standardowe i rozmiar maski.

9. Wykorzystując przygotowaną funkcję `fgaussian` stwórz maskę o rozmiarze $5 \times 5$ i odchyleniu standardowym 0.5.
  Wykorzystując funkcję `mesh` zwizualizuj filtr.
  Sprawdź jak parametr ``odchylenie standardowe'' wpływa na ``kształt'' filtru.

  Uwaga. W OpenCV dostępna jest *dedykowana* funkcja do filtracji Gaussa - `GaussianBlur`.
  Proszę na jednym przykładzie porównać jej działanie z użytym wyżej rozwiązaniem.

10. Wykonaj filtrację dla wybranych (2--3) wartości odchylenia standardowego.


In [ ]:
def fgaussian(size, sigma):
     m = n = size
     h, k = m//2, n//2
     x, y = np.mgrid[-h:h+1, -k:k+1]
     g = np.exp(-(x**2 + y**2)/(2*sigma**2))
     return g /g.sum() 
    
    
def mesh(fun, size):
    X = np.arange(-size//2, size//2, 1)
    Y = np.arange(-size//2, size//2, 1)
    X, Y = np.meshgrid(X, Y)
    Z = fun

    return X, Y, Z
    
    
    

In [ ]:
SIZES = [7, 15, 21, 25]
STD_DEV = 0
PATH = 'lena.png'
img = cv2.imread(PATH, cv2.IMREAD_GRAYSCALE)

for i, s in enumerate(SIZES):
    blur = cv2.GaussianBlur(img,(s,s), STD_DEV)
    plt.imshow(blur, 'gray')
    # plt.imshow(cv2.filter2D(img, -1, mask), 'gray')
    plt.axis('off')
    plt.show()


# Filtry nieliniowe -- mediana

Filtry rozmywające redukują szum, ale niekorzystnie wpływają na ostrość obrazu.
Dlatego często wykorzystuje się filtry nieliniowe - np. filtr medianowy (dla przypomnienia: mediana - środkowa wartość w posortowanym ciągu liczb).

Podstawowa różnica pomiędzy filtrami liniowymi, a nieliniowymi polega na tym, że przy filtracji liniowej na nową wartość piksela ma wpływ wartość wszystkich pikseli z otoczenia (np. uśrednianie, czasem ważone), natomiast w przypadku filtracji nieliniowej jako nowy piksel wybierana jest któraś z wartości otoczenia - według jakiegoś wskaźnika (wartość największa, najmniejsza czy właśnie mediana).


1. Wczytaj obraz _lenaSzum.png_ (losowe 10% pikseli białych lub czarnych - tzw. zakłócenia impulsowe). Przeprowadź filtrację uśredniającą z rozmiarem maski 3x3. Wyświetl, podobnie jak wcześniej, oryginał, wynik filtracji i moduł z różnicy. Wykorzystując funkcję ``cv2.medianBlur` wykonaj filtrację medianową _lenaSzum.png_ (z rozmiarem maski $3 \times 3$). Wyświetl, podobnie jak wcześniej, oryginał, wynik filtracji i moduł z różnicy. Która filtracja lepiej radzi sobie z tego typu szumem?

  Uwaga. Taki sam efekt da również użycie funkcji `signal.medfilt2d`.


In [ ]:

def mean_filter(img, kernel_size):
    kernel = np.ones((kernel_size, kernel_size), np.float32) / (kernel_size * kernel_size)
    filtered = cv2.filter2D(img, -1, kernel)

    fig, axs = plt.subplots(1, 3, figsize=(50,20))
    axs[0].imshow(img, 'gray')
    axs[0].axis('off')
    axs[0].set_title('Original', fontsize=40)

    axs[1].imshow(filtered, 'gray')
    axs[1].axis('off')
    axs[1].set_title('Filtered (mean)', fontsize=40)

    axs[2].imshow(np.abs(img - filtered), 'gray')
    axs[2].axis('off')
    axs[2].set_title('Substraction abs', fontsize=40)
    
    
    
def median_filter(img, kernel_size):
    filtered = cv2.medianBlur(img, kernel_size)

    fig, axs = plt.subplots(1, 3, figsize=(50,20))
    axs[0].imshow(img, 'gray')
    axs[0].axis('off')
    axs[0].set_title('Original', fontsize=40)

    axs[1].imshow(filtered, 'gray')
    axs[1].axis('off')
    axs[1].set_title('Filtered (median)', fontsize=40)

    axs[2].imshow(np.abs(img - filtered), 'gray')
    axs[2].axis('off')
    axs[2].set_title('Substraction abs', fontsize=40)
    
    
    
PATH = 'lenaSzum.png'
KERNEL_DIM = 3
img = cv2.imread(PATH, cv2.IMREAD_GRAYSCALE)

mean_filter(img, KERNEL_DIM)
median_filter(img, KERNEL_DIM)

2. Przeprowadź filtrację uśredniającą, a następnie medianową obrazu _lena.png_.
   Wyniki porównaj - dla obu wyświetl: oryginał, wynik filtracji i moduł z różnicy.
   Szczególną uwagę zwróć na ostrość i krawędzie.
   W której filtracji krawędzie zostają lepiej zachowane?

In [ ]:
PATH = 'lena.png'
KERNEL_DIM = 3
img = cv2.imread(PATH, cv2.IMREAD_GRAYSCALE)

mean_filter(img, KERNEL_DIM)
median_filter(img, KERNEL_DIM)

3. Ciekawy efekt można uzyskać wykonując filtrację medianową wielokrotnie. Określa się go mianem  posteryzacji.  W wyniku przetwarzania z obrazka usunięte zostają detale, a duże obszary uzyskują tą samą wartość jasności.  Wykonaj operację mediany $5 \times 5$ na obrazie _lena.png_ 10-krotnie. (wykorzystaj np. pętlę `for`).


Inne filtry nieliniowe:
- filtr modowy - moda (dominanta) zamiast mediany,
- filtr olimpijski - średnia z podzbioru otoczenia (bez wartości ekstremalnych),
- hybrydowy filtr medianowy - mediana obliczana osobno w różnych podzbiorach otoczenia (np. kształt ``x'',``+''), a jako wynik brana jest mediana ze zbioru wartość elementu centralnego, mediana z ``x'' i mediana z ``+'',
- filtr minimalny i maksymalny (będą omówione przy okazji operacji morfologicznych w dalszej części kursu).


Warto zdawać sobie sprawę, z szerokich możliwości dopasowywania rodzaju filtracji do konkretnego rozważanego problemu i rodzaju zaszumienia występującego na obrazie.

In [ ]:
def median_filter_posterization(img, kernel_size, iterations=10):
    filtered = cv2.medianBlur(img, kernel_size)
    for _ in range(iterations - 1):
        filtered = cv2.medianBlur(filtered, kernel_size)

    fig, axs = plt.subplots(1, 3, figsize=(50,20))
    axs[0].imshow(img, 'gray')
    axs[0].axis('off')
    axs[0].set_title('Original', fontsize=40)

    axs[1].imshow(filtered, 'gray')
    axs[1].axis('off')
    axs[1].set_title('Filtered (median, iterations = {0})'.format(iterations), fontsize=40)

    axs[2].imshow(np.abs(img - filtered), 'gray')
    axs[2].axis('off')
    axs[2].set_title('Substraction abs', fontsize=40)
    

PATH = 'lena.png'
KERNEL_DIM = 3
ITERATIONS = 10
img = cv2.imread(PATH, cv2.IMREAD_GRAYSCALE)

median_filter_posterization(img, KERNEL_DIM, ITERATIONS)

## Filtry liniowe górnoprzepustowe (wyostrzające, wykrywające krawędzie)

Zadaniem filtrów górnoprzepustowych jest wydobywanie z obrazu składników odpowiedzialnych za szybkie zmiany jasności - konturów, krawędzi, drobnych elementów tekstury.

### Laplasjan (wykorzystanie drugiej pochodnej obrazu)

1. Wczytaj obraz _moon.png_.

2. Wprowadź podstawową maskę laplasjanu:
\begin{equation}
M = 
\begin{bmatrix}
0 & 1& 0 \\ 1 & -4 & 1 \\ 0 & 1 & 0
\end{bmatrix}
\end{equation}

3. Przed rozpoczęciem obliczeń należy dokonać normalizacji maski - dla rozmiaru $3 \times 3$ podzielić każdy element przez 9.
   Proszę zwrócić uwagę, że nie można tu zastosować takiej samej normalizacji, jak dla filtrów dolnoprzepustowanych, gdyż skutkowałby to dzieleniem przez 0.

4. Wykonaj konwolucję obrazu z maską (`c2.filter2D`). Przed wyświetleniem, wynikowy obraz należy poddać normalizacji (występują ujemne wartości). Najczęściej wykonuje się jedną z dwóch operacji:
- skalowanie (np. poprzez dodatnie 128 do każdego z pikseli),
- moduł (wartość bezwzględna).

Wykonaj obie normalizacje. 
Na wspólnym wykresie wyświetl obraz oryginalny oraz przefiltrowany po obu normalizacjach. 

In [ ]:
def laplacian_filter(img, kernel):
    normalized_kernel = kernel / kernel.shape[0] ** 2 
    filtered = cv2.filter2D(img, -1, normalized_kernel)

    fig, axs = plt.subplots(1, 3, figsize=(50,20))
    axs[0].imshow(img, 'gray')
    axs[0].axis('off')
    axs[0].set_title('Original', fontsize=40)

    axs[1].imshow(filtered + 128, 'gray')
    axs[1].axis('off')
    axs[1].set_title('Filtered (laplacian+128)', fontsize=40)

    axs[2].imshow(np.abs(filtered), 'gray')
    axs[2].axis('off')
    axs[2].set_title('Filtered abs(laplacian)', fontsize=40)
    
    
KERNEL = np.array([[0, 1, 0], [1, -4, 1], [0, 1, 0]])    
PATH = 'moon.png'
img = cv2.imread(PATH, cv2.IMREAD_GRAYSCALE)

laplacian_filter(img, KERNEL)

7. Efekt wyostrzenia uzyskuje się po odjęciu/dodaniu (zależy do maski) rezultatu filtracji laplasjanowej i oryginalnego obrazu. Wyświetl na jednym wykresie: obraz oryginalny, sumę oryginału i wyniku filtracji oraz różnicę (bezwzględną) oryginału i wyniku filtracji.
 Uwaga. Aby uniknąć artefaktów, należy obraz wejściowy przekonwertować do formatu ze znakiem.



In [ ]:
def laplacian_filter2(img, kernel):
    normalized_kernel = kernel / kernel.shape[0] ** 2 
    filtered = cv2.filter2D(img, -1, normalized_kernel)

    fig, axs = plt.subplots(1, 3, figsize=(50,20))
    axs[0].imshow(img, 'gray')
    axs[0].axis('off')
    axs[0].set_title('Original', fontsize=40)

    axs[1].imshow(filtered + img, 'gray')
    axs[1].axis('off')
    axs[1].set_title('Filtered (filtered+orginal)', fontsize=40)

    axs[2].imshow(np.abs(img - filtered), 'gray')
    axs[2].axis('off')
    axs[2].set_title('Filtered abs(original-filtered)', fontsize=40)
    
KERNEL = np.array([[0, 1, 0], [1, -4, 1], [0, 1, 0]])    
PATH = 'moon.png'
img = cv2.imread(PATH, cv2.IMREAD_GRAYSCALE)

laplacian_filter2(img, KERNEL)

### Gradienty (wykorzystanie pierwszej pochodnej obrazu)

1. Wczytaj obraz _kw.png_. Stwórz odpowiednie maski opisane w kolejnych punktach i dokonaj filtracji.
2. Wykorzystując gradient Robertsa przeprowadź detekcję krawędzi - poprzez wykonanie konwolucji obrazu z daną maską:
\begin{equation}
R1 = \begin{bmatrix} 0 & 0 & 0 \\ -1 & 0 & 0 \\ 0 & 1 & 0 \end{bmatrix}   
R2 = \begin{bmatrix} 0 & 0 & 0 \\ 0 & 0 & -1 \\ 0 & 1 & 0 \end{bmatrix}
\end{equation}

Wykorzystaj stworzony wcześniej kod (przy laplasjanie) - dwie metody normalizacji oraz sposób wyświetlania.

3. Analogicznie przeprowadź detekcję krawędzi za pomocą gradientu Prewitta (pionowy i poziomy)
\begin{equation}
P1 = \begin{bmatrix} -1 & 0 & 1 \\ -1 & 0 & 1 \\ -1 & 0 & 1 \end{bmatrix}   
P2 = \begin{bmatrix} -1 & -1 & -1 \\ 0 & 0 & 0 \\ 1 & 1 & 1 \end{bmatrix}
\end{equation}

4. Podobnie skonstruowany jest gradient Sobela (występuje osiem masek, zaprezentowane są dwie ``prostopadłe''):
\begin{equation}
S1 = \begin{bmatrix} -1 & 0 & 1 \\ -2 & 0 & 2 \\ -1 & 0 & 1 \end{bmatrix}   
S2 = \begin{bmatrix} -1 & -2 & -1 \\ 0 & 0 & 0 \\ 1 & 2 & 1 \end{bmatrix}
\end{equation}

Przeprowadź detekcję krawędzi za pomocą gradientu Sobela. 

In [ ]:
def x_filter(img, kernel, name):
    normalized_kernel = kernel / kernel.shape[0] ** 2 
    filtered = cv2.filter2D(img, -1, normalized_kernel)

    fig, axs = plt.subplots(1, 3, figsize=(50,20))
    axs[0].imshow(img, 'gray')
    axs[0].axis('off')
    axs[0].set_title('Original', fontsize=40)

    axs[1].imshow(filtered + 128, 'gray')
    axs[1].axis('off')
    axs[1].set_title('Filtered ({0}+128)'.format(name), fontsize=40)

    axs[2].imshow(np.abs(filtered), 'gray')
    axs[2].axis('off')
    axs[2].set_title('Filtered abs({0})'.format(name), fontsize=40)
    
    
KERNELS = [
    ('R1', np.array([[0, 0, 0], [-1, 0, 0], [0, 1, 0]])),
    ('R2', np.array([[0, 0, 0], [0, 0, -1], [0, 1, 0]])),
    ('P1', np.array([[-1, 0, 1], [-1, 0, 1], [-1, 0, 1]])),
    ('P2', np.array([[-1, -1, 1], [0, 0, 0], [1, 1, 1]])),
    ('S1', np.array([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]])),
    ('S2', np.array([[-1, -2, 1], [0, 0, 0], [1, 2, 1]])),
]
PATH = 'kw.png'
img = cv2.imread(PATH, cv2.IMREAD_GRAYSCALE)

for k in KERNELS:
    x_filter(img, k[1], k[0])

5. Na podstawie dwóch ortogonalnych masek np. Sobela można stworzyć tzw. filtr kombinowany - pierwiastek kwadratowy z sumy kwadratów gradientów:
\begin{equation}
OW = \sqrt{(O * S1)^2 + (O * S2)^2}
\end{equation}
gdzie:  $OW$ - obraz wyjściowy, $O$ - obraz oryginalny (wejściowy), $S1,S2$ - maski Sobela, $*$ - operacja konwolucji.

Zaimplementuj filtr kombinowany.

Uwaga. Proszę zwrócić uwagę na konieczność zmiany formatu danych obrazu wejściowego - na typ znakiem



In [ ]:
def combined_filter_sqrt(img, kernel_1, kernel_2, name_1, name_2):
    
    normalized_kernel_1 = kernel_1 / kernel_1.shape[0] ** 2 
    normalized_kernel_2 = kernel_2 / kernel_2.shape[0] ** 2 
    filtered_1 = cv2.filter2D(img, -1, normalized_kernel_1)
    filtered_2 = cv2.filter2D(img, -1, normalized_kernel_2)
    
    result = np.sqrt(np.power(filtered_1, 2) + np.power(filtered_2, 2))
    result = result.astype('uint8')
    
    fig, axs = plt.subplots(1, 2, figsize=(50,20))
    axs[0].imshow(img, 'gray')
    axs[0].axis('off')
    axs[0].set_title('Original', fontsize=40)


    axs[1].imshow(result, 'gray')
    axs[1].axis('off')
    axs[1].set_title('Filtered sqrt((O * {0})^2 + (O * {1})^2 )'.format(name_1, name_2), fontsize=40)

    

PATH = 'kw.png'
img = cv2.imread(PATH, cv2.IMREAD_GRAYSCALE)

for i in range(0, len(KERNELS), 2):
    combined_filter_sqrt(img, KERNELS[i][1], KERNELS[i + 1][1], KERNELS[i][0], KERNELS[i + 1][0])

6. Istnieje alternatywna wersja filtra kombinowanego, która zamiast pierwiastka z sumy kwadratów wykorzystuje sumę modułów (prostsze obliczenia). 
Zaimplementuj tę wersję. 

In [ ]:
def combined_filter_abs(img, kernel_1, kernel_2, name_1, name_2):
    
    normalized_kernel_1 = kernel_1 / kernel_1.shape[0] ** 2 
    normalized_kernel_2 = kernel_2 / kernel_2.shape[0] ** 2 
    filtered_1 = cv2.filter2D(img, -1, normalized_kernel_1)
    filtered_2 = cv2.filter2D(img, -1, normalized_kernel_2)
    
    result = np.abs(filtered_1) + np.abs(filtered_2)
    result = result.astype('uint8')
    
    fig, axs = plt.subplots(1, 2, figsize=(50,20))
    axs[0].imshow(img, 'gray')
    axs[0].axis('off')
    axs[0].set_title('Original', fontsize=40)


    axs[1].imshow(result, 'gray')
    axs[1].axis('off')
    axs[1].set_title('Filtered abs(O * {0}) + abs(O * {1})'.format(name_1, name_2), fontsize=40)

    

PATH = 'kw.png'
img = cv2.imread(PATH, cv2.IMREAD_GRAYSCALE)

for i in range(0, len(KERNELS), 2):
    combined_filter_abs(img, KERNELS[i][1], KERNELS[i + 1][1], KERNELS[i][0], KERNELS[i + 1][0])

7. Wczytaj plik _jet.png_ (zamiast _kw.png_).
Sprawdź działanie obu wariantów filtracji kombinowanej.

In [ ]:
PATH = 'jet.png'
img = cv2.imread(PATH, cv2.IMREAD_GRAYSCALE)

for i in range(0, len(KERNELS), 2):
    combined_filter_sqrt(img, KERNELS[i][1], KERNELS[i + 1][1], KERNELS[i][0], KERNELS[i + 1][0])

for i in range(0, len(KERNELS), 2):
    combined_filter_abs(img, KERNELS[i][1], KERNELS[i + 1][1], KERNELS[i][0], KERNELS[i + 1][0])
